# Homework #7
### Juan Martin Morelli (N12699038)
### NYU Stern

#### Exercise 1: Homogenous Beliefs

We are going to work with risk-neutral asset pricing with dividends given by $D_{t}=d(x_{t})$, where the state process $\{X_{t}\}$ takes values from the countable state set $S$.

The transition probabibility is given by
\begin{equation}
q(x,y)\equiv P\{X_{t+1}=y|X_{t}=x\}\text{  with  }x,y\in S
\end{equation}

The equilibrium price is given by the functional equation:
\begin{equation}
p(x)=\beta\sum_{y\in S}[d(y)+p(y)]q(x,y)
\end{equation}
where $\beta\in(0,1)$ and $d$ is any function from $S$ to $(0,\infty)$.

The following blocks specify a Julia script for a value function iteration that solves for the equilibrium price (risk neutral) of the asset.

I first create a general type that will define a general structure for risk neutral asset pricing. I will restrict this type to be only for a discrete Markov process, but that could be modified. In principle, under this code, dividends could be a function of the state.

Note that we could have a multidimensional state space (i.e. several variables) but the way I am going to design the script is as if we have a vector of states. Then, we would have to consider the joint states as one entry of the vector. For example, if we have two state variables (a vector of values for each state variable), the kronecker product of the vectors would give me a new vector of joint states. In this way, I can reduce the dimensions of the problem, and the transition probability matrix is just two-dimensional.

In [3]:
type Asset_Price
    β::Float64 # risk neutral agent's discount factor
    q::Array{Float64,2} # Transition probability matrix.
    dfun::Function # Generic Function for Dividends
    tol::Float64 # Tolerance Level
    maxit::Int # Maximum iterations
    price::Array{Float64,2} # Prices (it is a value function)
end 

Define a function for the dividends process:

In [4]:
function dfun(ap::Asset_Price)
    dvals = [1.0 0.0 0.0]'
    return dvals
end

dfun (generic function with 1 method)

The following function "Tprice" takes as input the possible values for next period's dividends and prices, and computes the current price of the asset. It is a one step iteration, which updates the value on the type that changed.

In [5]:
function Tprice!(ap::Asset_Price,dvals::Array{Float64,2})
    ap.price = ap.β*ap.q*(dvals+ap.price)
    Void
end

Tprice! (generic function with 1 method)

Now I define the "iter" function. It iterates over the previous one step update. Once again, it updates the vlue on the type.

In [6]:
function iter!(ap::Asset_Price)
    resid = 1.0
    iteration = 0
    dvals = ap.dfun(ap)
    while resid>ap.tol
        p0 = ap.price
        if iteration>ap.maxit
            break
        else
            Tprice!(ap,dvals)
            resid = norm(ap.price-p0)
        end
    end
    Void
end

iter! (generic function with 1 method)

Now I define the main function which takes as input the matrix of probabilities, since it is the only thing that changes:

In [7]:
function main(q::Array{Float64,2})
    β=.98
    tol = 1.0e-10
    maxit = 10000    
    P0 = ones(3,1)    
    ap = Asset_Price(β,q,dfun,tol,maxit,P0)

    iter!(ap)
    return ap.price
end

main (generic function with 1 method)

Finally, I define the probability matrices and call the main function:

In [8]:
q1 = [[.971,.145,.0] [.029,.778,.508] [.0,.077,.492]]
q2 = [[.871,.145,.0] [.029,.778,.508] [.1,.077,.492]]

p1 = main(q1)
p2 = main(q2)

P = [p1 p2]

# Compute Unconditional Probabilities
# I use traspose of q to get left eigenvectors
# I checked that the first eigenvalue was the unity one (the one we need)

A,B = eig(q1')
Π1 = B[:,1]/sum(B[:,1])

A,B = eig(q2')
Π2 = B[:,1]/sum(B[:,1])

Π = [Π1 Π2]

print("The prices are:")
print("  
")
print(round(P,2))
print("

")
print("The unconditional probabilities are:")
print("
")
print(round(Π,3))

The prices are:  
[40.76 23.5
 35.96 21.1
 34.57 20.28]

The unconditional probabilities are:
[0.813 0.45
 0.163 0.401
 0.025 0.149]

*Intuition:*
Note that in the second case we have a positive probability for severe recession, conditional on being on a state of normal growth. Hence, conditional on being on the NG state, it is now much more likely that next period the asset will pay zero dividends (0.129 vs 0.029). Furthermore, the unconditional probability of receiving zero dividends increased from 0.188 to 0.55. That is why prices fell so much.

#### Exercise 2: Heterogenous Beliefs (Analytical)


Now the equilibrium condition is modified to:
\begin{equation}
p(x)=\beta\max_{i\in\{1,2\}}\left\{\sum_{y\in S}[d(y)+p(y)]q_{i}(x,y)\right\}\text{  with  }x\in S
\end{equation}

Equation (3) is just the fixed point from:
\begin{equation}
Tp(x)=\beta\max_{i\in\{1,2\}}\left\{\sum_{y\in S}[d(y)+p(y)]q_{i}(x,y)\right\}\text{  with  }x\in S
\end{equation}


**Fact (1)**: $\left|\max_{i}a_{i}|\leq\max_{i}|a_{i}\right|$

To show that the Bellman equation has a unique solution in $b\mathbb{R}^{S}$, I extend the previous proof. So, first I show that T maps $b\mathbb{R}^{S}$ into itself:

\begin{equation}
\begin{split}
|Tp(x)|&=\beta\left|\max_{i\in\{1,2\}}\left\{\sum_{y\in S}[d(y)+p(y)]q_{i}(x,y)\right\}\right| \\
& \leq\beta\max_{i\in\{1,2\}}\left|\left\{\sum_{y\in S}[d(y)+p(y)]q_{i}(x,y)\right\}\right| \\
&\leq\beta\left(\lVert d\rVert_{\infty}+\lVert p\rVert_{\infty}\right)
\end{split}
\end{equation}

where from line one to two I used Fact (1) and form line two to three I skept steps that are exactly the same as seen in exercise 1. Since this bound is uniform over all $x$, taking the supremum on the LHS yields $ \lVert Tp(x) \rVert _{\infty}<\infty$, so it is bounded.

Now  I show that it is a uniform contraction:

\begin{equation}
\begin{split}
\left|Tp(x)-Tp'(x)\right|=&\beta\left|\max_{i\in\{1,2\}}\left\{\sum_{y\in S}[p(y)-p'(y)]q_{i}(x,y)\right\}\right| \\
&\leq\beta\sum_{y\in S}\max_{i\in\{1,2\}}\left|p(y)-p'(y)\right|q_{i}(x,y) \\
&\leq\beta\left\|p-p'\right\|_{\infty}
\end{split}
\end{equation}

where the step from the first to the second line came from Fact (1) and the hint in the assignment.
Since $x\in S$ was arbitrary, we get:
\begin{equation}
\left\|Tp-Tp'\right\|_{\infty}\leq\beta\left\|p-p'\right\|_{\infty}
\end{equation}



***Annex: extra proof.***
I will now show the hint: $\left|\sup_{i}a_{i}-\sup_{i}b_{i}\right|\leq\sup_{i}\left|a_{i}-b_{i}\right|$.

It is based on the proof that $\left|\left\|x-y\right\|\right|\leq\left\|x-y\right\|$. I assume that the supremum are finite.


$a_{i}=\left[a_{i}-b_{i}\right]+b_{i}\leq\sup_{i}\left(a_{i}-b_{i}\right)+\sup_{i}\left(b_{i}\right)$.


Since this is for an arbitrary $a_{i}$, it is true that 

$\sup_{i}\left(a_{i}\right)-\sup_{i}\left(b_{i}\right)\leq\sup_{i}\left(a_{i}-b_{i}\right)\leq\sup_{i}\left|a_{i}-b_{i}\right|$



Similarly,

$\sup_{i}\left(b_{i}\right)-\sup_{i}\left(a_{i}\right)\leq\sup_{i}\left(b_{i}-a_{i}\right)\leq\sup_{i}\left|a_{i}-b_{i}\right|$

Hence,


$\left|\sup_{i}\left(a_{i}\right)-\sup_{i}\left(b_{i}\right)\right|\leq\sup_{i}\left|a_{i}-b_{i}\right|$

#### Exercise 3: Heteregenous beliefs (Computational)

For this part I just have to add a "max" operator in the previous code. Thus, I will not explain it part by part but rather insert all in one cell. Here I jsut do it for 2 agents, but it would be nice to have a general code for several agents.

In [2]:
type Asset_Price_Heterogenous
    β::Float64 # risk neutral agent's discount factor
    q1::Array{Float64,2} # Transition probability matrix of agent 1.
    q2::Array{Float64,2} # Transition probability matrix of agent 2.
    dfun_Heterogenous::Function # Generic Function for Dividends
    tol::Float64 # Tolerance Level
    maxit::Int # Maximum iterations
    price::Array{Float64,2} # Prices (it is a value function)
end 


function dfun_Heterogenous(aph::Asset_Price_Heterogenous)
    dvals = [1.0 0.0 0.0]'
    return dvals
end


function Tprice_Heterogenous!(aph::Asset_Price_Heterogenous,dvals::Array{Float64,2})
    aph.price = aph.β*max(aph.q1*(dvals+aph.price),aph.q2*(dvals+aph.price))
    Void
end


function iter_Heterogenous!(aph::Asset_Price_Heterogenous)
    resid = 1.0
    iteration = 0
    dvals = aph.dfun_Heterogenous(aph)
    while resid>aph.tol
        p0 = aph.price
        if iteration>aph.maxit
            break
        else
            Tprice_Heterogenous!(aph,dvals)
            resid = norm(aph.price-p0)
        end
    end
    Void
end


function main_Heterogenous(q1::Array{Float64,2},q2::Array{Float64,2})
    β=.98
    tol = 1.0e-10
    maxit = 10000    
    P0 = ones(3,1)    
    aph = Asset_Price_Heterogenous(β,q1,q2,dfun_Heterogenous,tol,maxit,P0)

    iter_Heterogenous!(aph)
    return aph.price
end


q1 = [[.971,.145,.0] [.029,.778,.508] [.0,.077,.492]]
q2 = [[.871,.145,.0] [.029,.778,.508] [.1,.077,.492]]

p_Heterogenous = main_Heterogenous(q1,q2)


3x1 Array{Float64,2}:
 40.7598
 35.9609
 34.572 

So, in the end the price of the asset will be given by the agent which is more optimistic, since he has a higher valuation for it. The asset pays the dividend in a state where the first agent finds more probable to occur.